In [1]:
import pinocchio as pin
import math
import time
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.utils import *
import pinocchio as pio
import eigenpy
from scipy.optimize import fmin_bfgs
import numpy as np

import casadi as ca

from pinocchio import casadi as cpin

# from pinocchio.visualize import GepettoVisualizer

import matplotlib.pyplot as plt

In [2]:
exampleRobotDataPath = '/home/n7student/Documents/Boulot/CNRS@CREATE/Codes/OC & IRL/Starting/3 - URDF et pinocchio/assets/'
urdf = exampleRobotDataPath + 'mon_robot.urdf'
robot = RobotWrapper.BuildFromURDF( urdf, [ exampleRobotDataPath, ] )
# robot.setVisualizer(GepettoVisualizer()) # AJOUT
robot.initViewer(loadModel=True)
NQ = robot.model.nq
NV = robot.model.nv

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
q = ca.SX.sym("q",NQ,1)
dq = ca.SX.sym("dq",NQ,1)

In [4]:
# Implémentation des fonctions nécessaires au calcul des coûts avec cpin

frame_id = robot.model.getFrameId("ee_link")


def vtip_fun(q):
    # cpin.forwardKinematics(cmodel, cdata, q)
    v_frame = pin.getFrameVelocity(robot.model, robot.data, frame_id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
    # NOTE : vérifier que ce sont bien les deux bonnes composantes de la vitesse
    return v_frame.linear[[0, 2]]

vtip = ca.Function('vtip', [q], [vtip_fun(q)])

vtip_val = vtip(q) 
vx_sq = ca.dot(vtip_val[0], vtip_val[0])
vy_sq = ca.dot(vtip_val[1], vtip_val[1])

vx_sq_fun = ca.Function("vx_sq_fun", [q], [vx_sq])
vy_sq_fun = ca.Function("vy_sq_fun", [q], [vy_sq])

# oMtool = robot.data.oMf[IDX_EE]
# oRtool = oMtool.rotation
# Jee = oRtool[:2,:2] @ (pin.computeFrameJacobian(robot.model,robot.data,np.array([q1_test[i],q2_test[i]]),IDX_EE)[:2,:2])

# jac_fun = ca.Function("jac_fun", [q], [cpin.computeJointJacobians(cmodel,cdata,q)])
# jac_fun = ca.Function("jac_fun", [q], [(robot.data.oMf[IDX_EE].rotation)[:2,:2] @ (cpin.computeFrameJacobian(cmodel, cdata, q, IDX_EE)[:2,:2])])
jac_fun = ca.Function("jac_fun", [q], [cpin.computeFrameJacobian(robot.model, robot.data, q, frame_id, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED)])

manipulability_fun = ca.Function("manipulability_fun", [q], [jac_fun(q) @ jac_fun(q).T])

cost_manipulability_fun = ca.Function("cost_manipulability_fun", [q], [-ca.log(ca.det(manipulability_fun(q)) + 1e-6)])

ArgumentError: Python argument types in
    pinocchio.pinocchio_pywrap_casadi.computeFrameJacobian(Model, Data, SX, int, ReferenceFrame)
did not match C++ signature:
    computeFrameJacobian(pinocchio::ModelTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> model, pinocchio::DataTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> {lvalue} data, Eigen::Matrix<casadi::Matrix<casadi::SXElem>, -1, 1, 0, -1, 1> q, unsigned long frame_id)
    computeFrameJacobian(pinocchio::ModelTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> model, pinocchio::DataTpl<casadi::Matrix<casadi::SXElem>, 0, pinocchio::JointCollectionDefaultTpl> {lvalue} data, Eigen::Matrix<casadi::Matrix<casadi::SXElem>, -1, 1, 0, -1, 1> q, unsigned long frame_id, pinocchio::ReferenceFrame reference_frame)

In [5]:
opti = ca.Opti()

# cadre du problème
# temporel
t_0 = 0
t_f = 1
N = 100  # nombre de points de discrétisation
dt = (t_f - t_0) / N

# longueurs
L_1 = 1
L_2 = 1
N_angles = 2

# variables à optimiser
q = opti.variable(N_angles, N)
q_fic = opti.variable(1, N)
dq = opti.variable(N_angles, N)

# déclaration de la matrice de poids
# NOTE : j'avais mis des np.ones au début, mais cela ne marchait pas, il faut essayer de rester avec des objets casadi
w_1 = ca.DM.ones((N_angles, N))
w_2 = ca.DM.zeros(N)
w_3 = ca.DM.zeros(N)
w_4 = ca.DM.ones(N)


# \Phi_1 : coût sur les vitesses angulaires
Phi_1 = dq**2
J_1 = w_1[0,:] @ Phi_1[0,:].T + w_1[1,:] @ Phi_1[1,:].T

# variables symboliques indépendantes pour un pas de temps
q_sym = ca.MX.sym("q", 2)


J_2 = 0
J_3 = 0
J_4 = 0
save_cost_manipulability = []
for t in range(N):
    q_t = ca.vertcat(q[0, t], q[1, t])
    # q_t = ca.vertcat(q[0, t], q[1, t], q_fic[0, t])
    # cpin.framesForwardKinematics(cmodel, cdata, q_t)
    J_2 += w_2[t] * vx_sq_fun(q_t)
    J_3 += w_3[t] * vy_sq_fun(q_t)
    J_4 += w_4[t] * cost_manipulability_fun(q_t)

# fonction que l'on cherche à minimiser
opti.minimize(J_1 + J_2 + J_3 + J_4)

# contraintes d’intégration
opti.subject_to(q_fic[0, N-1] == 0)
for t in range(N-1):
    opti.subject_to(q[:, t+1] == q[:, t] + dt * dq[:, t])
    opti.subject_to(q_fic[0, t] == 0)
opti.subject_to(q[:, 0] == [0, ca.pi/4])
opti.subject_to(L_1*ca.cos(q[0, -1]) + L_2*ca.cos(q[0, -1] + q[1, -1]) == -0.75)

# conditions supplémentaires sur les angles, non demandé dans l'énoncé
# opti.subject_to(opti.bounded(-pi/2, q, pi/2))

opti.solver("ipopt")
sol = opti.solve()

q1_pin = sol.value(q[0,:])
q2_pin = sol.value(q[1,:])

dq1_pin = sol.value(dq[0,:])
dq2_pin = sol.value(dq[1,:])

print("q1 =", q1_pin)
print("q2 =", q2_pin)
print("dq1 =", dq1_pin)
print("dq2 =", dq2_pin)

NameError: name 'cost_manipulability_fun' is not defined

In [ ]:
robot.display(np.array([q1_pin[0], q2_pin[0]]))

time.sleep(2)
for i in range(N):
    robot.display(np.array([q1_pin[i],q2_pin[i]]))
    time.sleep(0.1)